In [28]:
%pip install duckdb

In [29]:
import pandas as pd
import duckdb

In [30]:
# Create a DuckDB connection and read a CSV file into a DuckDB table
db = duckdb.connect()
db.execute(
    """
    CREATE TABLE transactions AS
    SELECT * FROM read_parquet('https://raw.githubusercontent.com/subwaymatch/mba564b-2025-redesign/main/03-SQL-database-to-dashboard/data/state-of-delaware-pcard-transactions.parquet');
"""
)

In [31]:
db.sql("SELECT * FROM transactions LIMIT 2;")

┌─────────────┬───────────────┬────────────────────┬────────────────────────┬───────────────────────┬────────────────────────────────────────────────────┬─────────────────────┬────────┐
│ fiscal_year │ fiscal_period │     department     │        division        │       merchant        │                category_description                │  transaction_date   │ amount │
│    int64    │     int64     │      varchar       │        varchar         │        varchar        │                      varchar                       │    timestamp_ns     │ double │
├─────────────┼───────────────┼────────────────────┼────────────────────────┼───────────────────────┼────────────────────────────────────────────────────┼─────────────────────┼────────┤
│        2017 │             1 │ LEGISLATIVE BRANCH │ General Assembly House │ VZWRLSS*APOCC VISB    │ Telecom Incl Prepaid-Recurring Phone Svcs          │ 2016-07-14 00:00:00 │  11.65 │
│        2017 │             1 │ LEGISLATIVE BRANCH │ General Assembly 

In [32]:
query_num_rows = db.sql("SELECT COUNT(*) AS num_rows FROM transactions;")
query_num_rows

┌──────────┐
│ num_rows │
│  int64   │
├──────────┤
│  1585108 │
└──────────┘

In [33]:
query_top_10_departments = db.sql(
    """SELECT department, SUM(amount) AS total_spent
        FROM transactions
        GROUP BY department
        ORDER BY total_spent DESC
        LIMIT 10;
"""
)

query_top_10_departments

┌────────────────────────────────┬────────────────────┐
│           department           │    total_spent     │
│            varchar             │       double       │
├────────────────────────────────┼────────────────────┤
│ DEPT OF CORRECTIONS            │ 213372349.32000133 │
│ DEPT OF TRANSPORTATION         │ 105867360.91000076 │
│ DEPT OF NATRL RES AND ENV CONT │  40121062.60000045 │
│ DEL TECH AND COMM COLLEGE      │  38555314.62000014 │
│ EXECUTIVE                      │  28860860.46999994 │
│ DEPT OF TECHNOLOGY AND INFOR   │ 27838467.730000023 │
│ DEPT OF SAFETY AND HOMELAND    │  22436893.33999991 │
│ DEPT OF HEALTH AND SOCIAL SV   │ 22131629.409999818 │
│ DEPT OF STATE                  │ 17263785.379999902 │
│ APPOQUINIMINK SCHOOL DISTRICT  │ 14194091.139999807 │
├────────────────────────────────┴────────────────────┤
│ 10 rows                                   2 columns │
└─────────────────────────────────────────────────────┘

In [34]:
query_2025 = db.sql(
    "SELECT * FROM transactions WHERE fiscal_year = 2025 AND amount > 10000;"
)
query_2025

┌─────────────┬───────────────┬────────────────────────────────┬────────────────────────────────┬─────────────────────────┬───────────────────────────────────────────────────┬─────────────────────┬──────────┐
│ fiscal_year │ fiscal_period │           department           │            division            │        merchant         │               category_description                │  transaction_date   │  amount  │
│    int64    │     int64     │            varchar             │            varchar             │         varchar         │                      varchar                      │    timestamp_ns     │  double  │
├─────────────┼───────────────┼────────────────────────────────┼────────────────────────────────┼─────────────────────────┼───────────────────────────────────────────────────┼─────────────────────┼──────────┤
│        2025 │             1 │ EXECUTIVE                      │ Fleet Management               │ MILFORD CHEVY OFFICE    │ Auto Truck Dlrs-Sales Svc Reprs Prts Lea